# Contents
* [load_data](#load_data)
* [evaluation](#evaluation)
* [train](#train)
* [cross_entropy](#cross_entropy)
* [histogram](#histogram)

In [1]:
import numpy as np
import os
import pandas as pd
import time
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision

from evaluation import *
from glob import glob
from losses import *
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from torch.autograd import Variable
from torch.optim import lr_scheduler
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset, DataLoader

In [2]:
seed = 18
torch.cuda.manual_seed_all(seed)
use_gpu = torch.cuda.is_available()
use_gpu

True

# load_data

In [3]:
class MarketDataset(Dataset):
    def __init__(self, paths, labels=None, label_encoder=None, transform=None):
        self.transform = transform
        self.img_paths = paths
        
        if label_encoder is not None:
            self.labels = torch.from_numpy(label_encoder.transform(labels))
        else:
            self.labels = None

    def __len__(self):
        return len(self.img_paths)

    def __getitem__(self, index):
        img_path = self.img_paths[index]
        img = Image.open(img_path)

        if self.transform:
            img = self.transform(img)

        if self.labels is not None:
            label = self.labels[index]
            return (img, label)
        else:
            return img

def create_train_df(x):
    df = pd.DataFrame({'path': x})
    df['label'] = df.path.apply(lambda x: int(x.split('/')[-1].split('_')[0]))

    return df

def create_test_df(x):
    df = create_train_df(x)
    df['camera'] = df.path.apply(lambda x: int(x.split('/')[-1].split('_')[1].split('s')[0].split('c')[1]))
    df['name'] = df.path.apply(lambda x: x.split('/')[-1])
    #df['is_junk'] = df['label'] == -1
    #df['is_distractor'] = df['label'] == 0
    
    return df

def create_df(x, is_train):
    if is_train:
        return create_train_df(x)
    else:
        return create_test_df(x)

In [4]:
MARKET_ROOT = 'data/Market-1501-v15.09.15'
# todo try equal transformers for train and test
data_transforms = {
    'train': transforms.Compose([
        transforms.Scale([256, 256]),
        transforms.RandomCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Scale([256, 256]),
        transforms.CenterCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}

info = {
    'train': {
        'path': 'bounding_box_train/*.jpg',
        'is_train': True,
        'transform': data_transforms['train'],
        'batch_size': 64,
        'label_encoder': None
    },
    'test': {
        'path': 'bounding_box_test/*.jpg',
        'is_train': False,
        'transform': data_transforms['test'],
        'batch_size': 32,
        'label_encoder': None
    },
    'query': {
        'path': 'query/*.jpg',
        'is_train': False,
        'transform': data_transforms['test'],
        'batch_size': 32,
        'label_encoder': None
    }
}

dfs = {
    x: create_df(sorted(glob(os.path.join(MARKET_ROOT, info[x]['path']))), info[x]['is_train']) 
    for x in info.keys()
}

for key in dfs.keys():
    print(key, len(dfs[key]))
    dfs[key].to_csv('%s.csv' % key, index=None, header=None)
    
train_cl = np.array([int(x.split('/')[-1].split('_')[0]) for x in dfs['train']['path']])
train_label_encoder = preprocessing.LabelEncoder()
train_label_encoder.fit(train_cl)
train_cl_n = train_label_encoder.classes_.shape[0]

info['train']['label_encoder'] = train_label_encoder

image_datasets = {
    x: MarketDataset(dfs[x]['path'], dfs[x]['label'], 
                     info[x]['label_encoder'], info[x]['transform']) 
    for x in info.keys()
}
dataset_size = len(image_datasets['train'])
dataloders = {
    x: torch.utils.data.DataLoader(image_datasets[x], batch_size=info[x]['batch_size'],
                                      shuffle=info[x]['is_train'], num_workers=10)
    for x in info.keys()
}

train 12936
test 19732
query 3368


# evaluation

In [5]:
def get_descr(dataloder, model):
    result = []
    
    for data in dataloder:
        if use_gpu:
            inputs = Variable(data.cuda())
        else:
            inputs = Variable(data)

        out = model(inputs)
        result.extend(out.data.squeeze().cpu().numpy())
        
    return np.array(result)

def get_info(df):
    labels = np.array(df['label'])
    camera_labels = np.array(df['camera'])
    image_paths = np.array(df['path'])
    image_names = np.array(df['name'])
    
    return labels, camera_labels, image_paths, image_names

def make_index():
    Index = dict()
    Index['junk'] = set()
    Index['distractor'] = set()
    for name in test_image_names:
        if ifJunk(name):
            Index['junk'].add(name)
        elif ifDistractor(name):
            Index['distractor'].add(name)
    for query in query_image_names:
        Index[query] = dict()
        Index[query]['pos'] = set()
        Index[query]['junk'] = set()

        person, camera = parse_market_1501_name(query)
        for name in test_image_names:
            if ifJunk(name) or ifDistractor(name):
                continue
            person_, camera_ = parse_market_1501_name(name)
            if person == person_ and camera != camera_ :
                Index[query]['pos'].add(name)

            elif person == person_ and camera == camera :
                Index[query]['junk'].add(name)
                
    return Index

def ranks_map(model, dataloders, nr):
    model_without_fc = nn.Sequential(*list(model.children())[:-1])
    gescr_gallery = get_descr(dataloders['test'], model_without_fc)
    gescr_query = get_descr(dataloders['query'], model_without_fc)

    ranks, img_names_sorted, places = ranking('cosine', gescr_query, query_image_names, gescr_gallery, test_image_names, nr, Index)
    ranks_w = ranks / len(gescr_query) * 1.0
    mAP_ = mAP(gescr_query, query_image_names, gescr_gallery, test_image_names, nr, Index)
    
    return ranks_w, mAP_

In [6]:
test_labels, test_camera_labels, test_image_paths, test_image_names = get_info(dfs['test'])
query_labels, query_camera_labels, query_image_paths, query_image_names = get_info(dfs['query'])
Index = make_index()

# train

### cross_entropy

In [7]:
def train_model(model, criterion, optimizer, scheduler, num_epochs):
    since = time.time()

    for epoch in range(1, num_epochs + 1):
        print('Epoch %d/%d' % (epoch, num_epochs))
        print('-' * 10)
        
        scheduler.step()
        model.train(True)

        running_loss = .0
        running_corrects = 0

        for data in dataloders['train']:
            inputs, labels = data

            if use_gpu:
                inputs = Variable(inputs.cuda())
                labels = Variable(labels.cuda())
            else:
                inputs, labels = Variable(inputs), Variable(labels)

            optimizer.zero_grad()

            outputs = model(inputs)
            _, preds = torch.max(outputs.data, 1)
            loss = criterion(outputs, labels)

            loss.backward()
            optimizer.step()

            running_loss += loss.data[0]
            running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_size
            epoch_acc = running_corrects / dataset_size

        print('Loss: %.4f Acc: %.4f' % (epoch_loss, epoch_acc))

        model.train(False)
        
        if epoch % 5 == 0:
            ranks, mAP = ranks_map(model, dataloders, 50)
            print('rank1: %.4f mAP: %.4f' % (ranks[1], mAP))
        
        if epoch % 10 == 0:
            torch.save(model, 'models/finetuned_resnet_market_cross_entropy_e%d.pt' % epoch)
        
    time_elapsed = time.time() - since
    print('Training complete in %.0fm %.0fs' % (
        time_elapsed // 60, time_elapsed % 60))

    return model

In [8]:
model = models.resnet50(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = torch.nn.Sequential()
model.fc.add_module('dropout', torch.nn.Dropout(p=0.75))
model.fc.add_module('fc1', nn.Linear(num_ftrs, train_cl_n))

if use_gpu:
    model = model.cuda()
    
model

ResNet (
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
  (relu): ReLU (inplace)
  (maxpool): MaxPool2d (size=(3, 3), stride=(2, 2), padding=(1, 1), dilation=(1, 1))
  (layer1): Sequential (
    (0): Bottleneck (
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True)
      (relu): ReLU (inplace)
      (downsample): Sequential (
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True)
      )
    )
    (1): Bott

In [9]:
params_dict = dict(model.named_parameters())
params = []
for key, value in params_dict.items():
    print(key)
    if ('conv1' in key) & (not 'layer' in key):
        params += [{'params':[value],'lr':0.00001}]
    elif ('conv' in key):
        params += [{'params':[value],'lr':0.002}]
    elif ('fc1' in key):
        params += [{'params':[value],'lr':0.1}]
    elif 'bias' in key:
        params += [{'params':[value], 'lr':0.0001, 'weight_decay':0}]
    else:
        params += [{'params':[value], 'lr':0.0001}]

conv1.weight
bn1.weight
bn1.bias
layer1.0.conv1.weight
layer1.0.bn1.weight
layer1.0.bn1.bias
layer1.0.conv2.weight
layer1.0.bn2.weight
layer1.0.bn2.bias
layer1.0.conv3.weight
layer1.0.bn3.weight
layer1.0.bn3.bias
layer1.0.downsample.0.weight
layer1.0.downsample.1.weight
layer1.0.downsample.1.bias
layer1.1.conv1.weight
layer1.1.bn1.weight
layer1.1.bn1.bias
layer1.1.conv2.weight
layer1.1.bn2.weight
layer1.1.bn2.bias
layer1.1.conv3.weight
layer1.1.bn3.weight
layer1.1.bn3.bias
layer1.2.conv1.weight
layer1.2.bn1.weight
layer1.2.bn1.bias
layer1.2.conv2.weight
layer1.2.bn2.weight
layer1.2.bn2.bias
layer1.2.conv3.weight
layer1.2.bn3.weight
layer1.2.bn3.bias
layer2.0.conv1.weight
layer2.0.bn1.weight
layer2.0.bn1.bias
layer2.0.conv2.weight
layer2.0.bn2.weight
layer2.0.bn2.bias
layer2.0.conv3.weight
layer2.0.bn3.weight
layer2.0.bn3.bias
layer2.0.downsample.0.weight
layer2.0.downsample.1.weight
layer2.0.downsample.1.bias
layer2.1.conv1.weight
layer2.1.bn1.weight
layer2.1.bn1.bias
layer2.1.conv2.we

In [10]:
criterion = nn.CrossEntropyLoss()
optimizer_ft = optim.SGD(params, momentum=0.9, weight_decay=0.0001)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=40, gamma=0.1)

In [11]:
model = train_model(model, criterion, optimizer_ft,
                         exp_lr_scheduler, num_epochs=50)

Epoch 1/50
----------
Loss: 0.1036 Acc: 0.0372
Epoch 2/50
----------
Loss: 0.0673 Acc: 0.1885
Epoch 3/50
----------
Loss: 0.0446 Acc: 0.3849
Epoch 4/50
----------
Loss: 0.0319 Acc: 0.5261
Epoch 5/50
----------
Loss: 0.0238 Acc: 0.6320
rank1: 0.6336 mAP: 0.4005
Epoch 6/50
----------
Loss: 0.0183 Acc: 0.7058
Epoch 7/50
----------
Loss: 0.0143 Acc: 0.7655
Epoch 8/50
----------
Loss: 0.0131 Acc: 0.7826
Epoch 9/50
----------
Loss: 0.0135 Acc: 0.7823
Epoch 10/50
----------
Loss: 0.0102 Acc: 0.8284
rank1: 0.7001 mAP: 0.4658
Epoch 11/50
----------
Loss: 0.0084 Acc: 0.8613
Epoch 12/50
----------
Loss: 0.0069 Acc: 0.8819
Epoch 13/50
----------
Loss: 0.0059 Acc: 0.8970
Epoch 14/50
----------
Loss: 0.0050 Acc: 0.9088
Epoch 15/50
----------
Loss: 0.0040 Acc: 0.9307
rank1: 0.7248 mAP: 0.4817
Epoch 16/50
----------
Loss: 0.0041 Acc: 0.9258
Epoch 17/50
----------
Loss: 0.0031 Acc: 0.9423
Epoch 18/50
----------
Loss: 0.0027 Acc: 0.9547
Epoch 19/50
----------
Loss: 0.0032 Acc: 0.9459
Epoch 20/50
-------